# Read in LGA list and clean up for use in google api loops

In [1]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup as bs
# Google API Key
from config import gkey

#file path to saved csv dataframe
file = 'Raw_Data/Vic-LGA-List.csv'

In [ ]:
# Establishing a connection to postgreSQL
import psycopg2
from sqlalchemy import create_engine
from entrancekey import postgresqlkey
from sqlalchemy import create_engine  
from sqlalchemy import Column, String  
from sqlalchemy.ext.declarative import declarative_base  
from sqlalchemy.orm import sessionmaker

In [2]:
#bring csv file in as pd dataframe
council_city_df = pd.read_csv(file, encoding="ISO-8859-1")
council_city_df

,ï»¿COUNCIL,CEO NAME,DEAR CEO,CEO TITLE,MAYOR NAME,DEAR MAYOR,MAYOR TITLE,POSTAL,POSTAL SUBURB,PCODE,DX,ADDRESS,SUBURB/ TOWN,CODE,TEL,FAX,EMAIL,WWW SITE
0,Alpine Shire Council,Mr Charlie Bird,Mr Bird,Chief Executive Officer,Cr Peter Roper,Cr Roper,Mayor,PO Box 139,BRIGHT,3741,,Great Alpine Road,BRIGHT,3741,5755 0555,5755 1811,info@alpineshire.vic.gov.au,www.alpineshire.vic.gov.au
1,Ararat Rural City Council,Dr Tim Harrison,Dr Harrison,Chief Executive Officer,Cr Jo Armstrong,Cr Armstrong,Mayor,PO Box 246,ARARAT,3377,,59 Vincent Street,ARARAT,3377,5355 0200,5352 1695,council@ararat.vic.gov.au,www.ararat.vic.gov.au
2,Ballarat City Council,Ms Janet Dore,Ms Dore,Chief Executive Officer,Cr Ben Taylor,Cr Taylor,Mayor,PO Box 655,BALLARAT,3353,,25 Armstrong Street South,BALLARAT,3350,5320 5500,5333 4061,info@ballarat.vic.gov.au,www.ballarat.vic.gov.au
3,Banyule City Council,Ms Allison Beckwith,Ms Beckwith,Chief Executive Officer,Cr Alison Champion,Cr Champion,Mayor,PO Box 94,GREENSBOROUGH,3088,DX 97904 Ivanhoe,1 Flintoff Street,GREENSBOROUGH,3088,9490 4222,9499 9475,enquiries@banyule.vic.gov.au,www.banyule.vic.gov.au
4,Bass Coast Shire Council,Ms Ali Wastie,Ms Wastie,Chief Executive Officer,Cr Brett Tessari,Cr Tessari,Mayor,PO Box 118,WONTHAGGI,3995,DX 34903 Wonthaggi,76 McBride Avenue,WONTHAGGI,3995,1300 BCOAST (226278) or 5671 2211,5671 2222,basscoast@basscoast.vic.gov.au,www.basscoast.vic.gov.au
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Wodonga City Council,Mr Mark Dixon,Mr Dixon,Chief Executive Officer,Cr Anna Speedie,Cr Speedie,Mayor,PO Box 923,WODONGA,3689,,104 Hovell Street,WODONGA,3690,02 6022 9300,02 6022 9322,info@wodonga.vic.gov.au,www.wodonga.vic.gov.au
75,Wyndham City Council,Ms Kelly Grigsby,Ms Grigsby,Chief Executive Officer,Cr Josh Gilligan,Cr Gilligan,Mayor,PO Box 197,WERRIBEE,3030,,45 Princes Highway,WERRIBEE,3030,9742 0777,9741 6237,mail@wyndham.vic.gov.au,www.wyndham.vic.gov.au
76,Yarra City Council,Ms Vijaya Vaidyanath,Ms Vaidyanath,Chief Executive Officer,Cr Misha Coleman,Cr Coleman,Mayor,PO Box 168,RICHMOND,3121,DX 30205 Richmond,333 Bridge Road,RICHMOND,3121,9205 5555,,info@yarracity.vic.gov.au,www.yarracity.vic.gov.au
77,Yarra Ranges Shire Council,Ms Tammi Rose,Ms Rose,Chief Executive Officer,Cr Richard Higgins,Cr Higgins,Mayor,PO Box 105,LILYDALE,3140,DX 34051 Lilydale,Anderson Street,LILYDALE,3140,1300 368 333,9735 4249,mail@yarraranges.vic.gov.au,www.yarraranges.vic.gov.au


In [3]:
print('The number of Local Government Areas (councils) in Vic is: '+str(len(council_city_df)))

The number of Local Government Areas (councils) in Vic is: 79


In [4]:
# Get list of column headings for cleaning up df
council_city_df.columns.unique()

Index(['ï»¿COUNCIL', 'CEO NAME', 'DEAR CEO', 'CEO TITLE', 'MAYOR NAME',
       'DEAR MAYOR', 'MAYOR TITLE', 'POSTAL', 'POSTAL SUBURB', 'PCODE', 'DX',
       'ADDRESS', 'SUBURB/ TOWN', 'CODE', 'TEL', 'FAX', 'EMAIL', 'WWW SITE'],
      dtype='object')

In [5]:
# Clean council_city_df to have just the columns wanted
council_city_df=council_city_df.drop(['CEO NAME', 'DEAR CEO', 'CEO TITLE', 
                                      'MAYOR NAME','DEAR MAYOR', 'MAYOR TITLE', 
                                      'POSTAL', 'POSTAL SUBURB', 'PCODE', 'DX',
                                      'ADDRESS','TEL', 'FAX', 'EMAIL'], axis=1)
# Re-name council column
council_city_df = council_city_df.rename(columns={"ï»¿COUNCIL": "COUNCIL"})
council_city_df

,COUNCIL,SUBURB/ TOWN,CODE,WWW SITE
0,Alpine Shire Council,BRIGHT,3741,www.alpineshire.vic.gov.au
1,Ararat Rural City Council,ARARAT,3377,www.ararat.vic.gov.au
2,Ballarat City Council,BALLARAT,3350,www.ballarat.vic.gov.au
3,Banyule City Council,GREENSBOROUGH,3088,www.banyule.vic.gov.au
4,Bass Coast Shire Council,WONTHAGGI,3995,www.basscoast.vic.gov.au
...,...,...,...,...
74,Wodonga City Council,WODONGA,3690,www.wodonga.vic.gov.au
75,Wyndham City Council,WERRIBEE,3030,www.wyndham.vic.gov.au
76,Yarra City Council,RICHMOND,3121,www.yarracity.vic.gov.au
77,Yarra Ranges Shire Council,LILYDALE,3140,www.yarraranges.vic.gov.au


In [6]:
# Make city list
cities = council_city_df["SUBURB/ TOWN"].tolist()
print(cities)

['BRIGHT', 'ARARAT', 'BALLARAT', 'GREENSBOROUGH', 'WONTHAGGI', 'DROUIN', 'SANDRINGHAM', 'BENALLA', 'CAMBERWELL', 'SUNSHINE', 'WYCHEPROOF', 'ECHUCA', 'OFFICER', 'NARRE WARREN', 'MARYBOROUGH', 'COLAC', 'CAMPERDOWN', 'PRESTON', 'BAIRNSDALE', 'FRANKSTON', 'KERANG', 'CAULFIELD SOUTH', 'PORTLAND', 'BANNOCKBURN', 'BENDIGO', 'DANDENONG', 'GEELONG', 'SHEPPARTON', 'DAYLESFORD', 'NHILL', 'ALTONA', 'HORSHAM', 'BROADMEADOWS', 'BEECHWORTH', 'CHELTENHAM', 'WANTIRNA SOUTH', 'MORWELL', 'WEDDERBURN', 'KYNETON', 'DONCASTER', 'MANSFIELD', 'FOOTSCRAY', 'RINGWOOD', 'MELBOURNE', 'MELTON', 'MILDURA', 'BROADFORD', 'COBRAM', 'GLEN WAVERLEY', 'MOONEE PONDS', 'BALLAN', 'COBURG', 'ROSEBUD', 'CASTLEMAINE', 'PORT FAIRY', 'ALEXANDRA', 'GREENSBOROUGH', 'STAWELL', 'ST KILDA', 'BEAUFORT', 'QUEENSCLIFF', 'LEONGATHA', 'HAMILTON', 'MALVERN', 'EUROA', 'TORQUAY', 'SWAN HILL', 'TALLANGATTA', 'WANGARATTA', 'WARRNAMBOOL', 'SALE', 'EDENHOPE', 'NUNAWADING', 'SOUTH MORANG', 'WODONGA', 'WERRIBEE', 'RICHMOND', 'LILYDALE', 'WARRACKNA

In [7]:
cities_list = np.array(cities)
print(cities_list)

['BRIGHT' 'ARARAT' 'BALLARAT' 'GREENSBOROUGH' 'WONTHAGGI' 'DROUIN'
 'SANDRINGHAM' 'BENALLA' 'CAMBERWELL' 'SUNSHINE' 'WYCHEPROOF' 'ECHUCA'
 'OFFICER' 'NARRE WARREN' 'MARYBOROUGH' 'COLAC' 'CAMPERDOWN' 'PRESTON'
 'BAIRNSDALE' 'FRANKSTON' 'KERANG' 'CAULFIELD SOUTH' 'PORTLAND'
 'BANNOCKBURN' 'BENDIGO' 'DANDENONG' 'GEELONG' 'SHEPPARTON' 'DAYLESFORD'
 'NHILL' 'ALTONA' 'HORSHAM' 'BROADMEADOWS' 'BEECHWORTH' 'CHELTENHAM'
 'WANTIRNA SOUTH' 'MORWELL' 'WEDDERBURN' 'KYNETON' 'DONCASTER' 'MANSFIELD'
 'FOOTSCRAY' 'RINGWOOD' 'MELBOURNE' 'MELTON' 'MILDURA' 'BROADFORD'
 'COBRAM' 'GLEN WAVERLEY' 'MOONEE PONDS' 'BALLAN' 'COBURG' 'ROSEBUD'
 'CASTLEMAINE' 'PORT FAIRY' 'ALEXANDRA' 'GREENSBOROUGH' 'STAWELL'
 'ST KILDA' 'BEAUFORT' 'QUEENSCLIFF' 'LEONGATHA' 'HAMILTON' 'MALVERN'
 'EUROA' 'TORQUAY' 'SWAN HILL' 'TALLANGATTA' 'WANGARATTA' 'WARRNAMBOOL'
 'SALE' 'EDENHOPE' 'NUNAWADING' 'SOUTH MORANG' 'WODONGA' 'WERRIBEE'
 'RICHMOND' 'LILYDALE' 'WARRACKNABEAL']


# Get Tourist Attractions using loop through cities and calling google api

In [8]:
# use google api to search things to do for each city

#lists to append to
name_ta = []
address_ta = []
rating_ta = []
city_ta = []
city_searched = []
for city in cities_list:
    citi = city
    city_ta.append(citi)
    my_phrase = f"tourist attraction establishment in {city} VIC, AU"
    target_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
    "query": my_phrase,
    "key": gkey
    }
    response = requests.get(target_url, params)
    to_dos = response.json()
    counter = 0
    # print the response URL, avoid doing for public GitHub repos in order to avoid exposing key
    print("----------------------")
    #print(response.url)
    #print(json.dumps(to_dos, indent=4, sort_keys=True))
    for do in to_dos["results"]:
        counter += 1
        try:
            print("-------------------")
            print(f'{citi}')
            city_searched.append(citi)
            called = do["name"]
            locate = do["formatted_address"]
            rated = do["rating"]
            if(called, locate and rated):
                print("-------------------")
                name_ta.append(called)
                print(called)
                address_ta.append(locate)
                print(locate)
                rating_ta.append(rated)
                print(rated)
        except:
            print('not found')           
    
    if counter == 20:
        time.sleep(1.01)
        counter = 0


e VIC 3852, Australia
4.8
-------------------
SALE
-------------------
Cunninghame Street Walking/Cycling Path
2 Cunninghame St, Sale VIC 3850, Australia
4.3
-------------------
SALE
-------------------
Sale and Districts Aeromodellers Club
Centre Rd, Sale VIC 3850, Australia
4.9
-------------------
SALE
-------------------
Gippsland Vehicle Collection
1 Maffra-Sale Rd, Maffra VIC 3860, Australia
4.6
-------------------
SALE
-------------------
Maffra Visitor Information Centre
96 Johnson St, Maffra VIC 3860, Australia
4.2
----------------------
-------------------
EDENHOPE
-------------------
Edenhope Information Centre
96 Elizabeth St, Edenhope VIC 3318, Australia
4.8
-------------------
EDENHOPE
-------------------
Edenhope N.F.R.
Edenhope VIC 3318, Australia
0
-------------------
EDENHOPE
-------------------
Sheepwash, Edenhope W.R
Kaniva-Edenhope Rd, Edenhope VIC 3318, Australia
0
-------------------
EDENHOPE
-------------------
Kadnook Bushland Reserve
Edenhope VIC 3318, Australi

# Create and clean df from tourist attraction data gathered 

In [12]:
print("Total returned Tourist Attractions: "+ str(len(name_ta)))

Total returned Tourist Attractions: 1573


In [20]:
# Create a df from tourise attractions
tourist_site_df = pd.DataFrame(name_ta)
tourist_site_df["tourist_site_address"] = pd.Series(address_ta)
tourist_site_df["tourist_site_rating"] = pd.Series(rating_ta)
tourist_site_df["tourist_site_city"] = pd.Series(city_searched) # To overcome the issues of lenght not matching index


In [22]:
# Rename column 0 tourist_sites_df
tourist_site_df = tourist_site_df.rename(columns={0: "tourist_site_name"})
tourist_site_df

,tourist_site_name,tourist_site_address,tourist_site_rating,tourist_site_city
0,Huggins Lookout Point,"Huggins Rd, Bright VIC 3741, Australia",4.4,BRIGHT
1,Howitt Park,"7 Riverside Ave, Bright VIC 3741, Australia",4.6,BRIGHT
2,Apex Lookout,"Bright VIC 3741, Australia",4.4,BRIGHT
3,Bright Centenary Park,"Canyon Walk, Bright VIC 3741, Australia",4.7,BRIGHT
4,Tower Hill Lookout,"Bright VIC 3741, Australia",4.4,BRIGHT
...,...,...,...,...
1568,Werrigar Roadhouse & Motel,"213 Henty Hwy, Warracknabeal VIC 3393, Australia",4.3,WARRACKNABEAL
1569,Lutheran Church of Australia,"235 Scott St, Warracknabeal VIC 3393, Australia",3.6,WARRACKNABEAL
1570,Warracknabeal Early Learning Centre,"11 Werrigar St, Warracknabeal VIC 3393, Australia",0.0,WARRACKNABEAL
1571,The Creekside Hotel Warracknabeal,"4 Scott St, Warracknabeal VIC 3393, Australia",4.4,WARRACKNABEAL


In [23]:
#drop duplicates in tourist_sites_df
tourist_site_xdbl_df=tourist_site_df.drop_duplicates(subset=['tourist_site_name', 'tourist_site_city'],keep='last')
tourist_site_xdbl_df

,tourist_site_name,tourist_site_address,tourist_site_rating,tourist_site_city
0,Huggins Lookout Point,"Huggins Rd, Bright VIC 3741, Australia",4.4,BRIGHT
1,Howitt Park,"7 Riverside Ave, Bright VIC 3741, Australia",4.6,BRIGHT
2,Apex Lookout,"Bright VIC 3741, Australia",4.4,BRIGHT
3,Bright Centenary Park,"Canyon Walk, Bright VIC 3741, Australia",4.7,BRIGHT
4,Tower Hill Lookout,"Bright VIC 3741, Australia",4.4,BRIGHT
...,...,...,...,...
1568,Werrigar Roadhouse & Motel,"213 Henty Hwy, Warracknabeal VIC 3393, Australia",4.3,WARRACKNABEAL
1569,Lutheran Church of Australia,"235 Scott St, Warracknabeal VIC 3393, Australia",3.6,WARRACKNABEAL
1570,Warracknabeal Early Learning Centre,"11 Werrigar St, Warracknabeal VIC 3393, Australia",0.0,WARRACKNABEAL
1571,The Creekside Hotel Warracknabeal,"4 Scott St, Warracknabeal VIC 3393, Australia",4.4,WARRACKNABEAL


In [24]:
# save csv of tourist sites
tourist_site_xdbl_df.to_csv("Cleaned_data/tourist_sites.csv", encoding="utf-8", index=False)